<a href="https://colab.research.google.com/github/TurkuNLP/Deep_Learning_in_LangTech_course/blob/master/laser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install laserembeddings
!python -m laserembeddings download-models


✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fcodes    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fvocab    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/bilstm.93langs.2018-12-26.pt    

✨ You're all set!


In [2]:
from laserembeddings import Laser

laser = Laser()
#can this be any simpler? :)
embeddings = laser.embed_sentences(['I love pasta.',"J'adore les pâtes.",'Ich liebe Pasta.'],lang=['en', 'fr', 'de'])

print(embeddings)
print(embeddings.shape)


[[-5.2039034e-04 -2.8321798e-05 -1.6871556e-04 ...  3.4788880e-03
  -1.9968916e-03  8.1148157e-03]
 [ 3.2193225e-03 -9.9815712e-05  5.9067454e-05 ...  7.6490263e-03
   1.1962595e-03  2.4502855e-03]
 [ 5.3412135e-04 -3.6210098e-05 -1.4794555e-04 ...  6.1386474e-03
  -1.6569832e-03  6.3126395e-03]]
(3, 1024)


# Test the embeddings

* We are working on a paraphrase corpus, from which I borrowed some early data
* The two files below `yle.txt` and `hs.txt` contain some 200+ news titles from YLE and HS, judged by a human to be paraphrases or near-paraphrases of each other
* The selection is such that lexical overlap is minimized
* The two files are line-aligned
* We could make a simple test of LASER, comparing them against each other to see if we can pair these up
* In other words: for every HS title, find the nearest YLE title
* Measure how often it is correct
* Random baseline is roughly 1/200, i.e. about 0.5%

In [3]:
!wget -nc http://dl.turkunlp.org/.ginter/hs.txt
!wget -nc http://dl.turkunlp.org/.ginter/yle.txt

File ‘hs.txt’ already there; not retrieving.

File ‘yle.txt’ already there; not retrieving.



In [4]:
def read_file(fname):
  lines=[]
  with open(fname) as f:
    for line in f:
      line=line.strip()
      if not line:
        continue
      lines.append(line)
  return lines

hs=read_file("hs.txt")
yle=read_file("yle.txt")

hs_vectors=laser.embed_sentences(hs,"fi")
yle_vectors=laser.embed_sentences(yle,"fi")

print("hs",hs_vectors.shape)
print("yle",yle_vectors.shape)

hs (217, 1024)
yle (217, 1024)


In [5]:
import sklearn.metrics
# Given two sets of vectors, this function calculates all-pair cosine distances
all_dist=sklearn.metrics.pairwise_distances(hs_vectors,yle_vectors)
print("Distance matrix shape:", all_dist.shape)
#we get a sentence-by-sentence matrix, with distances

Distance matrix shape: (217, 217)


In [6]:
#Calculate for every row the document with minimal distance in that row (axis=-1 means minimum along the last axis)
nearest=all_dist.argmin(axis=-1) #These are the nearest neighbors for each HS title (indices into YLE), perfect solution would be [0,1,2,3...,216]
print(nearest)

[121   1   1   3 120 116   6   9   8   9 177  11 102  13  44  15 126  17
 196 206 189  96  22  23  70   9  77 189 216  67  30 136  78  61  34  74
  53  37 161  39  40 147 203  13  44  45   9  47  48  49  14  51 120  53
  54  55 121  46  58 123  60  61 189  48  64  65  39  13 126  69  70  71
  61  73  74  61  76  26 189  74  14  81  82  83  84  85  86  87 182  64
  15 135 176 189  70  59 206  97 208  99  99  84 102 103 104 105  48  67
  91 175 110  14 112 113  39 115 116 121 118  73 120 121 121 123 124 125
 126 159 128 129 130  92 132 133 134  61 136 137 138   9  39 141 142 123
  39   9  35  15 150 149  67 151 152 153 180 155 156 137 158  79 160 176
  79 163 164 112  39 167 168 169 132 171 172 146  61 175 201 177 178 179
 180  63 189 183 184  67  39 187 197 189  94 146 192 193 150 195 196 197
  78  39 198  74 202 203 204 133 206   9 208 146  86  59  30 213 214 215
 216]


In [0]:
# Let's package this all nicely into a function
import random

def eval_embeddings(texts1,texts2,vectors1,vectors2):
  assert len(texts1)==len(texts2), "We assume aligned data"
  all_dist=sklearn.metrics.pairwise_distances(vectors1,vectors2)
  nearest=all_dist.argmin(axis=-1) #These are the nearest neighbors for each HS title (indices into YLE), perfect solution would be [0,1,2,3...,216]   
  correct=[] #Let's put here the correct pairs
  incorrect=[] #Let's put here the incorrect pairs
  for i,txt1 in enumerate(texts1):
    j=nearest[i] #the index at which the nearest sentence is
    txt2=texts2[j] #..and its text
    if i==j:
      #This is correct
      correct.append((txt1,txt2))
    else:
      incorrect.append((txt1,txt2))

  print(f"Correct {len(correct)}/{len(texts1)}={len(correct)/len(texts1)*100}%") #these f-strings are really neat, you can embed expressions and have them printed
  random.shuffle(correct)
  random.shuffle(incorrect)
  print("\n\n---------- Sample of correct ones:")
  for t1,t2 in correct[:15]:
    print(t1)
    print(t2)
    print()
  print("\n\n---------- Sample of incorrect ones:\n")
  for t1,t2 in incorrect[:15]:
    print(t1)
    print(t2)
    print()

In [8]:
eval_embeddings(hs,yle,hs_vectors,yle_vectors)

Correct 110/217=50.69124423963134%


---------- Sample of correct ones:
Tutkimus: Lapsen riski sairastua astmaan on sitä pienempi, mitä enemmän kotona on bakteereja
Kodin sisäilman mikrobisto voi suojella lasta astmalta

Lännen Media: EU ei myönnä kriisiapua maatalouden kuivuus­ongelmaan, koska rahaa sen hoitamiseen ei ole
Euroopan unionilta ei ole tulossa kriisitukea maataloudelle

Presidentti Niinistö leikattu Jorvin sairaalassa – Toimenpide sujui hyvin, potilas on jo kotiutettu
Presidentti Niinistö pääsi päiväkirurgisesta leikkauksesta – viikon matkustuskielto

Varas ajoi holtittomasti läpi Töölön – Pakomatkalla kolhittiin pahoin kuutta autoa ja jalankulkijat olivat vaarassa jäädä alle
Hurja takaa-ajo Helsingissä – lapsi työnnettiin kumoon kassajonossa, jalankulkijat hyppivät sivuun poliisia paenneen auton tieltä

Pääministeri Sipilä Oulun epäillystä seksuaali­rikoksesta: ”Oikeusvaltiossa syylliset saavat rangaistuksen etnisyydestä riippumatta”
Pääministeri Sipilä Oulun raiskausepäi

# Try with BERT?

*   We could try with BERT
*   Test the [CLS] token as the sentence embedding
*   Test the average of token embeddings as the sentence embedding



In [9]:
#Note: since LASER is torch, maybe we continue in torch for the fun of it? :) (and you also asked for some torch examples)
!pip install transformers
import transformers

bert_model = transformers.BertModel.from_pretrained("bert-base-finnish-cased-v1") #models can be loaded by name from this list: https://github.com/huggingface/transformers/blob/master/src/transformers/modeling_bert.py#L35
bert_model = bert_model.cuda() #move the model to GPU
bert_model.eval() #tell the model it will be used for predictions, not training (disables dropout for example)


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(50105, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [10]:
import torch
import torch.nn

#Load the Finnish BERT tokenizer
tokenizer = transformers.BertTokenizer.from_pretrained("bert-base-finnish-cased-v1") #also tokenizers can be loaded by nae

def tokenize_texts(texts):
  tokenized_ids=[tokenizer.encode(txt,add_special_tokens=True) for txt in texts] #this runs the BERT tokenizer, returns list of lists of integers
  tokenized_ids_t=[torch.tensor(ids,dtype=torch.long) for ids in tokenized_ids] #turn lists of integers into torch tensors
  tokenized_single_batch=torch.nn.utils.rnn.pad_sequence(tokenized_ids_t,batch_first=True) #zero-padding
  return tokenized_single_batch

hs_data=tokenize_texts(hs).cuda() #tokenize and move to GPU
yle_data=tokenize_texts(yle).cuda()

print(hs_data.shape)

torch.Size([217, 37])


In [11]:
#This is how you run BERT in torch
data=hs_data
with torch.no_grad(): #tell the model not to gather gradients since we are evaluating, not training, saves memory and troubles
  emb=bert_model(data.cuda()) #applies the model and returns several things, we care about the first. Documentation: https://github.com/huggingface/transformers/blob/master/src/transformers/modeling_bert.py#L648
  print(emb[0].shape)  # word x sequence x embedding


torch.Size([217, 37, 768])


In [12]:
#Let's pack this into a nice function
def embed(data,how_to_pool="CLS"):
  with torch.no_grad(): #tell the model not to gather gradients
    emb=bert_model(data.cuda()) #runs BERT and returns several things, we care about the first
    #emb[0]  # batch x word x embedding
    if how_to_pool=="AVG":
      pooled=emb[0].mean(1) #average along the word dimension (axis 1) -> pools the embeddings into one
    elif how_to_pool=="CLS":
      pooled=emb[0][:,0,:].squeeze() #Pick the first token as the embedding
    else:
      assert False, "how_to_pool should be CLS or AVG"
    print("Pooled shape:",pooled.shape)
  return pooled.cpu().numpy() #done! move data back to CPU and extract the numpy array

hs_emb_cls=embed(hs_data,"CLS")
yle_emb_cls=embed(yle_data,"CLS")

hs_emb_avg=embed(hs_data,"AVG")
yle_emb_avg=embed(yle_data,"AVG")


Pooled shape: torch.Size([217, 768])
Pooled shape: torch.Size([217, 768])
Pooled shape: torch.Size([217, 768])
Pooled shape: torch.Size([217, 768])


In [13]:
eval_embeddings(hs,yle,hs_emb_cls,yle_emb_cls)

Correct 25/217=11.52073732718894%


---------- Sample of correct ones:
Nordea sai uuden konserni­johtajan, kurssi nousussa – Suuromistaja Gardell vaatii kannattavuuden parantamista nopeasti ja ”dramaattisesti”
Nordea löysi omistaan uuden konsernijohtajan – Frank Vang-Jensen aloittaa tehtävässä jo tänään

Etla: Työpaikan menetys lisää ihmisen muutto­alttiutta
Etlan tutkimus: Työttömyys pistää muuttokuorman herkästi liikkeeseen

Li Andersson toivoo opetusministerin salkkua, sisäministerin paikka menossa vihreille – Tämä tiedetään salkkujaosta
Vuorossa salkkujako – puolueiden puheenjohtajat koolle ratkomaan ministeripaikkoja

Palvelunesto­hyökkäys on kaatanut esimerkiksi poliisin ja Verohallinnon verkko­sivut
Poliisin ja Verohallinnon verkkosivuilla on taas häiriöitä – syy ei vielä tiedossa

Tutkimus: 90 prosenttia Suomessa asuvista somaleista kokee tulevansa syrjityksi työmarkkinoilla
Pääkaupunkiseudun maahanmuuttajat kokevat kuuluvansa suomalaiseen yhteiskuntaan, mutta suomalainen ident

In [14]:
eval_embeddings(hs,yle,hs_emb_avg,yle_emb_avg)

Correct 53/217=24.42396313364055%


---------- Sample of correct ones:
Ministeri Lindström pani nimensä paperiin, jonka merkitystä ei täysin ymmärtänyt – Päätös oli viedä Suomen valtion oikeuskamppailuun miljoonista euroista
Ministeri Lindström oli vähällä viedä valtion miljoonakorvauksiin talousrikolliselle – "Allekirjoitukseni saatiin viekkaudella"

Poliisi saanut useita kymmeniä uusia vihjeitä liittyen lasten törkeistä hyväksikäytöistä epäiltyyn ”Enoon”: Joukossa mahdollisia uhreja
Poliisi epäilee "Enoksi" kutsuttua miestä useista lapseen kohdistuneista seksuaalirikoksista Helsingissä – ainakin 12 uhria

Suositusta olkapää­leikkauksesta pitäisi luopua kokonaan, sanovat asiantuntijat – osa kirurgeista uskoo yhä turhaksi todistetun toimen­piteen hyötyyn
Asiantuntijat: Olkapään avarrusleikkaukset ovat terveydenhuollon voimavarojen tuhlaamista – ei ole hyötyä kivusta kärsiville

Suojelupoliisi osallistuu Airiston Helmen kotietsintä­aineiston tarkasteluun – valtava aineisto vastaa jopa 1

# What have we learned?

* BERT [CLS] embedding is the worst
* Average of BERT token embeddings comes then
* LASER still doubles the numbers
* What kind of intuitive insight one gets when looking at the correct and incorrect predictions? What does the model base its decisions on?

